In [2]:
integral_count = 1000

def integral_by_trapezoidal_rule(f, start_x, end_x):
    sum = 0
    width = (end_x - start_x) / integral_count
    for i in range(integral_count):
        area = width * (f(start_x + i*width) + f(start_x + (i + 1)*width)) / 2
#         if i==500:
#             print(area)
        sum += area
    
    return sum


# # 積分関数正しいかチェック
# area = integral_by_trapezoidal_rule(y, 0, 30)
# print(area)

# def y(x):
#     return x

In [3]:
# 方程式を定義
import numpy as np
import math
import functools

def func_r(q):
    value = q / (1 - b + b * q) ** 2
    return value


def func_g_integrand(z, m, q):
    integrand = (1 / math.sqrt(2 * np.pi)) * math.exp(-z**2/2) * np.tanh(b * (m + math.sqrt(a * func_r(q)) * z)) ** 2
    return integrand

    
def func_f_integrand(z, m, q):
    integrand =  (1 / math.sqrt(2 * np.pi)) * math.exp(-z**2/2) * np.tanh(b * (m + math.sqrt(a * func_r(q)) * z))
    return integrand
    

def func_g(m, q):
    func = functools.partial(func_g_integrand, m=m, q=q)
    value =  integral_by_trapezoidal_rule(func,  -1000, 1000)
    return value
    
    
def func_f(m, q):
    func = functools.partial(func_f_integrand, m=m, q=q)
    value = integral_by_trapezoidal_rule(func,  -1000, 1000)
    return value


# ニュートン法の更新式を定義

# def differential_of_sqrt_r(q):
#     value_1 = 1 / (2 * math.sqrt(r(q)) )
#     denominator_of_value_2 = 1 - 2*q * (b-1-q) - (b-1-q)**2
#     numerator_of_value_2 = (1 - (b-1-q)**2 )**2
#     return value_1 * numerator_of_value_2 / denominator_of_value_2
    

# # j11
# def j_11_integrand(z, m, q):
#     integrand =  (1 / math.sqrt(2 * np.pi)) * math.exp(-z**2/2) / np.cosh(b * (m + math.sqrt(a * r(q)) * z))**2    * b
#     return integrand
    
# def j_11(m, q):
#     func = functools.partial(j_11_integrand, m=m, q=q)
#     return 1 - integral_by_trapezoidal_rule(func, -100, 100)    


# # j12
# def j_12_integrand(z, m, q):
#     integrand =  (1 / math.sqrt(2 * np.pi)) * math.exp(-z**2/2) / np.cosh(b * (m + math.sqrt(a * r(q)) * z))**2    * math.sqrt(a)*z* differential_of_sqrt_r(q)
#     return integrand
    
# def j_12(m, q):
#     func = functools.partial(j_12_integrand, m=m, q=q)
#     return 1 - integral_by_trapezoidal_rule(func, -100, 100)

    
# # j21
# def j_21_integrand(z, m, q):
#     integrand =  (1 / math.sqrt(2 * np.pi)) * math.exp(-z**2/2) / np.cosh(b * (m + math.sqrt(a * r(q)) * z))**2    * b    * 2*np.tanh(b * (m + math.sqrt(a * r(q)) * z))
#     return integrand
    
# def j_21(m, q):
#     func = functools.partial(j_21_integrand, m=m, q=q)
#     return 1 - integral_by_trapezoidal_rule(func, -100, 100)
    

# # j22
# def j_22_integrand(z, m, q):
#     integrand =  (1 / math.sqrt(2 * np.pi)) * math.exp(-z**2/2) / np.cosh(b * (m + math.sqrt(a * r(q)) * z))**2    * math.sqrt(a)*z* differential_of_sqrt_r(q)    *2*np.tanh(b * (m + math.sqrt(a * r(q)) * z))
#     return integrand
    
# def j_22(m, q):
#     func = functools.partial(j_22_integrand, m=m, q=q)
#     return 1 - integral_by_trapezoidal_rule(func, -100, 100)


# # calculate det
# def get_matrics_info(m, q):
#     j11 = j_11(m, q)
#     j12 = j_12(m, q)
#     j21 = j_21(m, q)
#     j22 = j_22(m, q)
#     return {"det": j11*j22 - j12*j21, "matrics":[[0, 0, 0], [0, j11, j12], [0, j21, j22]]}

In [4]:
# m, q のプロット

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

a = 0.1
b = 3

def plot_f_and_g(plot_num, zlim, args):
    fig = plt.figure(figsize = (8, 8))
    ax = fig.add_subplot(111, projection='3d', zlim=(-1.0, 1.0))
    ax.set_title("f(m,q) & g(m,q)", size = 20)

    ax.set_xlabel("m", size = 20)
    ax.set_ylabel("q", size = 20)
    ax.set_zlabel("", size = 20)

    x = np.linspace(args["m_min"], args["m_max"], plot_num)
    y = np.linspace(args["q_min"], args["q_max"], plot_num)

    X, Y = np.meshgrid(x, y)
    X = X.flatten()
    Y = Y.flatten()
    X, Y
    F = np.ndarray([])
    G = np.ndarray([])
    for i in range(len(Y)-1):
        #print(func_f(X[i], Y[i]))
        F = np.append(F, func_f(X[i], Y[i]))
        G = np.append(G, func_g(X[i], Y[i]))


    ax.plot(X, Y, F, color = "red", label='f (m,q)')
    ax.plot(X, Y, G, color = "green", label='g (m,q)')

    # これ呼ばないとラベルでない
    plt.legend()

    plt.show()

In [5]:
'''
常磁性層　　　　　　　　　　：　q = 0, m = 0
スピングラス層　　： q > 0, m = 0
想起層　　　　　　　　　　　　　： q > 0, m > 0
'''

# a = 0.01
# b = 4

# args = {
#     "m_min": -3,
#     "m_max": 3,
#     "q_min": 0,
#     "q_max": 3,
# }
# plot_f_and_g(plot_num=40, zlim=(-1.0, 1.0), args=args)

'\n常磁性層\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000：\u3000q = 0, m = 0\nスピングラス層\u3000\u3000： q > 0, m = 0\n想起層\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000： q > 0, m > 0\n'

In [6]:
# a = 0.01
# b = 0.1

# args = {
#     "m_min": -3,
#     "m_max": 3,
#     "q_min": 0,
#     "q_max": 3,
# }
# plot_f_and_g(plot_num=40, zlim=(-1.0, 1.0), args=args)

In [7]:
import numpy as np

In [11]:
'''
① a,bをメッシュに区切る
② 特定のa,bにおいて、m,qを0~1でメッシュに区切る
③ m,qのメッシュ全てでf,gを計算する
④ f,gがともに0に一番近いときのm,qを答えとして採用
'''

A = np.linspace(0, 1, 5)
B = np.linspace(0, 5, 5)

# ①のメッシュ
a_arr, b_arr = np.meshgrid(A, B)
a_arr = a_arr.flatten()
b_arr = b_arr.flatten()

answer_m = np.ndarray([])
answer_q = np.ndarray([])
for i in range(len(a_arr)):
    print("i:",i, "番目です")
    M = np.linspace(0, 1, 5)
    Q = np.linspace(0, 1, 5)
    
    # ②のメッシュ
    m, q = np.meshgrid(M, Q)
    m = m.flatten()
    q = q.flatten()

    m_i,q_i
    d = 100000
    a = a_arr[i]
    b = b_arr[i]
    
    # ③,④
    for j in range(len(m)):
        f = func_f(m[i], q[i])
        g = func_g(m[i], q[i])
        tmp_d = f**2 + g**2
        if tmp_d < d:
            d = tmp_d
            m_i = m[i]
            q_i = q[i]
            
    answer_m = np.append(answer_m, m_i)
    answer_q = np.append(answer_q, q_i)

print(answer_m)
print(answer_q)

i: 0 番目です
i: 1 番目です
i: 2 番目です
i: 3 番目です
i: 4 番目です
i: 5 番目です
i: 6 番目です
i: 7 番目です
i: 8 番目です
i: 9 番目です
i: 10 番目です
i: 11 番目です
i: 12 番目です
i: 13 番目です
i: 14 番目です
i: 15 番目です
i: 16 番目です
i: 17 番目です
i: 18 番目です
i: 19 番目です
i: 20 番目です
i: 21 番目です
i: 22 番目です
i: 23 番目です
i: 24 番目です
[0.   0.   0.25 0.5  0.75 1.   0.   0.25 0.5  0.75 1.   0.   0.25 0.5
 0.75 1.   0.   0.25 0.5  0.75 1.   0.   0.25 0.5  0.75 1.  ]
[5.   0.   0.   0.   0.   0.   0.25 0.25 0.25 0.25 0.25 0.5  0.5  0.5
 0.5  0.5  0.75 0.75 0.75 0.75 0.75 1.   1.   1.   1.   1.  ]
